In [15]:
#Importing all libraries
import requests
from bs4 import BeautifulSoup
import csv
import collections
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
#From Github; how to replace and expand contactions
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

In [ ]:
#Headers will make it look like you are using a web browser
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
#We will use the iteration to retrieve and scrape the web pages, reviews, and ratings from each page on Yelp
reviews = []
ratings = [] 

neg_reviews = []
pos_reviews = []

review_corpus = []

df = pd.read_csv('Supercuts Review List.csv')

for i, row in df.iterrows():
    if row['Brand'] == 'Great Clips': 
        url = row['url']
        response = requests.get(url, headers=headers, verify=False).text
        soup = BeautifulSoup(response, "lxml")
    #Looping through 'div' 'review-content' will help find all the review containers we need in each page that have rating and review
        for s in soup.find_all("div", attrs={'class': 'review-content'}):
            re = s.find('p', attrs={'lang': 'en'})
    #Makes all the letters lower in reviews
            review = re.text.lower()
            review_corpus.append([review])
    #expandContractions will put the dictionary made earlier to replace the contractions in the reviews
    #Make sure to to run the cList dict cell or else there will be an error
            expandContractions(review)
    #Cleaning the lemmas or words in reviews now will make it easier when we start predictive modeling
            words = word_tokenize(review)
            words = word_tokenize(review.replace('\n',' '))
            clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
            characters_to_remove = ["''",'``','...']
            clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
            english_stops = set(stopwords.words('english'))
            clean_words = [word for word in clean_words if word not in english_stops]
            wordnet_lemmatizer = WordNetLemmatizer()
            lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
            reviews.append(lemma_list)
    #Here we are using a simple control flow to recode the ratings for our model. If rating is greater than 3 positive, else negative   
            rating = s.find_all('img', attrs={'class': 'offscreen'})
    #the rating is actually an image, so we need to convert it into a string and then to an integer
            rate = str(rating)
            int_rating = int(rate[11:12])

            if int_rating == 1 or int_rating == 2 or int_rating == 3:
                rating = 'neg'
                ratings.append('neg')
                neg_reviews.append(lemma_list)
            elif int_rating == 4 or int_rating == 5:
                rating = 'pos'
                ratings.append('pos')
                pos_reviews.append(lemma_list)

my_df = pd.DataFrame(review_corpus)

my_df.to_csv('haircut_review_corpus_greatclips.csv', index=False, header=False)

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

In [ ]:
data_words_pos = []

data_words_neg = []

for wd_list in pos_reviews:
    for word in wd_list:
        data_words_pos.append(word)
        
for wd_list in neg_reviews:
    for word in wd_list:
        data_words_neg.append(word)

In [ ]:
data_words_pos = [data_words_pos]

data_words_neg = [data_words_neg]

In [ ]:
# Build the bigram and trigram models
bigram_pos = gensim.models.Phrases(data_words_pos, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_pos = gensim.models.Phrases(bigram_pos[data_words_pos], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_pos = gensim.models.phrases.Phraser(bigram_pos)
trigram_mod_pos = gensim.models.phrases.Phraser(trigram_pos)

In [ ]:
bigram_neg = gensim.models.Phrases(data_words_neg, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_neg = gensim.models.Phrases(bigram_neg[data_words_neg], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_neg = gensim.models.phrases.Phraser(bigram_neg)
trigram_mod_neg = gensim.models.phrases.Phraser(trigram_neg)

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams_pos(texts):
    return [bigram_mod_pos[doc] for doc in texts]

def make_trigrams_pos(texts):
    return [trigram_mod_pos[bigram_mod_pos[doc]] for doc in texts]

def make_bigrams_neg(texts):
    return [bigram_mod_neg[doc] for doc in texts]

def make_trigrams_neg(texts):
    return [trigram_mod_neg[bigram_mod_neg[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# Remove Stop Words
data_words_pos_nostops = remove_stopwords(data_words_pos)

data_words_neg_nostops = remove_stopwords(data_words_neg)

# Form Bigrams
data_words_bigrams_pos = make_bigrams_pos(data_words_pos_nostops)

data_words_bigrams_neg = make_bigrams_neg(data_words_neg_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_pos = lemmatization(data_words_bigrams_pos, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized_neg = lemmatization(data_words_bigrams_neg, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized_pos)

# Create Corpus
texts = data_lemmatized_pos

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

id2word_neg = corpora.Dictionary(data_lemmatized_neg)

# Create Corpus
texts = data_lemmatized_neg

# Term Document Frequency
corpus_neg = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [12]:
import pprint

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word_neg,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print('\n\n\n')
pprint.pprint(lda_model_neg.print_topics())
doc_lda_neg = lda_model_neg[corpus]

[(0,
  '0.001*"cut" + 0.001*"hair" + 0.001*"great" + 0.001*"go" + 0.001*"get" + '
  '0.001*"haircut" + 0.001*"time" + 0.001*"place" + 0.001*"want" + '
  '0.001*"clip"'),
 (1,
  '0.002*"hair" + 0.002*"great" + 0.002*"cut" + 0.002*"get" + 0.001*"haircut" '
  '+ 0.001*"go" + 0.001*"time" + 0.001*"good" + 0.001*"clip" + 0.001*"job"'),
 (2,
  '0.030*"cut" + 0.027*"hair" + 0.027*"get" + 0.025*"great" + 0.018*"time" + '
  '0.018*"haircut" + 0.015*"go" + 0.014*"good" + 0.011*"stylist" + '
  '0.011*"place"'),
 (3,
  '0.002*"hair" + 0.002*"great" + 0.002*"cut" + 0.001*"haircut" + 0.001*"get" '
  '+ 0.001*"come" + 0.001*"good" + 0.001*"time" + 0.001*"go" + 0.001*"job"'),
 (4,
  '0.038*"hair" + 0.030*"cut" + 0.028*"great" + 0.019*"haircut" + 0.019*"get" '
  '+ 0.018*"go" + 0.013*"time" + 0.013*"come" + 0.013*"good" + 0.010*"clip"'),
 (5,
  '0.002*"cut" + 0.002*"great" + 0.001*"hair" + 0.001*"haircut" + 0.001*"get" '
  '+ 0.001*"time" + 0.001*"go" + 0.001*"good" + 0.001*"come" + 0.001*"want"'),
 (6

In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.196056  0.017448       1        1  74.387360
2     -0.184486 -0.018911       2        1  25.612286
0      0.074768  0.000334       3        1   0.000056
9      0.072583  0.000671       4        1   0.000055
7      0.064789 -0.000910       5        1   0.000049
3      0.063868  0.000204       6        1   0.000048
5      0.059543 -0.000698       7        1   0.000045
1      0.056080 -0.000401       8        1   0.000043
8      0.005802  0.000289       9        1   0.000026
6     -0.016892  0.001975      10        1   0.000024, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
1069  Default  857.000000        hair  857.000000  30.0000  30.0000
601   Default  720.000000         cut  720.000000  29.0000  29.0000
1047  Default  666.000000       great  666.000000  28.0000  28.0000
1016  Default  521.000000         get  521.000000  27.0000  27.0000
1071  Default  463.000000     haircut  463.000000  26.0000  26.0000
1030  Default  411.000000          go  411.000000  25.0000  25.0000
2515  Default  353.000000        time  353.000000  24.0000  24.0000
481   Default  277.000000        come  277.000000  23.0000  23.0000
1034  Default  317.000000        good  317.000000  22.0000  22.0000
462   Default  239.000000        clip  239.000000  21.0000  21.0000
2391  Default  248.000000     stylist  248.000000  20.0000  20.0000
80    Default  214.000000      always  214.000000  19.0000  19.0000
1823  Default  244.000000       place  244.000000  18.0000  18.0000
1291  Default  207.000000         job  207.000000  17.0000  17.0000
2675  Default  206.000000        wait  206.000000  16.0000  16.0000
2684  Default  204.000000        want  204.000000  15.0000  15.0000
211   Default  190.000000        back  190.000000  14.0000  14.0000
416   Default  167.000000       check  167.000000  13.0000  13.0000
1491  Default  170.000000        make  170.000000  12.0000  12.0000
1433  Default  162.000000    location  162.000000  11.0000  11.0000
1023  Default  146.000000        give  146.000000  10.0000  10.0000
977   Default  159.000000    friendly  159.000000   9.0000   9.0000
1456  Default  140.000000        love  140.000000   8.0000   8.0000
2436  Default  134.000000        take  134.000000   7.0000   7.0000
1441  Default  133.000000        look  133.000000   6.0000   6.0000
853   Default  117.000000  experience  117.000000   5.0000   5.0000
2769  Default  143.000000        year  143.000000   4.0000   4.0000
1650  Default  125.000000        nice  125.000000   3.0000   3.0000
1713  Default  122.000000      online  122.000000   2.0000   2.0000
2187  Default  120.000000     service  120.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
601   Topic10    0.000052         cut  720.925781  -1.2122  -4.7313
1016  Topic10    0.000043         get  521.024719  -1.0853  -4.9292
1047  Topic10    0.000037       great  666.301147  -1.4955  -5.0934
481   Topic10    0.000024        come  277.458282  -1.0578  -5.5318
2515  Topic10    0.000022        time  353.493896  -1.3469  -5.5787
2684  Topic10    0.000018        want  204.824371  -1.0409  -5.8184
1071  Topic10    0.000024     haircut  463.075348  -1.5683  -5.5301
462   Topic10    0.000018        clip  239.380112  -1.1941  -5.8157
1030  Topic10    0.000022          go  411.536407  -1.5337  -5.6135
1034  Topic10    0.000018        good  317.513947  -1.4412  -5.7803
80    Topic10    0.000015      always  214.090317  -1.2270  -5.9603
2675  Topic10    0.000015        wait  206.609406  -1.2326  -6.0015
853   Topic10    0.000012  experience  117.585228  -0.8918  -6.2243
1023  Topic10    0.000013        give  146.188416  -1.0410  -6.1557
1291  Topic10    0.000014         job  207.677017  -1.2725  -6.0362
2436  Topic10    0.000012        take  134.95

In [14]:
vis_neg = pyLDAvis.gensim.prepare(lda_model_neg, corpus_neg, id2word_neg)
vis_neg

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x             y  topics  cluster       Freq
topic                                                    
3     -0.258387 -3.556811e-08       1        1  99.999649
0      0.054192  2.496155e-05       2        1   0.000047
9      0.046704 -6.228350e-05       3        1   0.000044
2      0.046195 -3.865488e-05       4        1   0.000044
7      0.034878  9.448232e-06       5        1   0.000040
4      0.033428  1.589156e-04       6        1   0.000040
6      0.027635 -2.296064e-05       7        1   0.000038
1      0.026290 -1.690878e-05       8        1   0.000038
8      0.018610 -5.774330e-05       9        1   0.000036
5     -0.029546  5.261333e-06      10        1   0.000033, topic_info=     Category         Freq             Term        Total  loglift  logprob
term                                                                      
1069  Default  1260.000000           easily  1260.000000  30.0000  30.0000
601   Default  1090.000000           circus  1090.000000  29.0000  29.0000
1047  Default   997.000000            dryer   997.000000  28.0000  28.0000
1016  Default   815.000000           doozie   815.000000  27.0000  27.0000
1071  Default   694.000000             easy   694.000000  26.0000  26.0000
1030  Default   611.000000             drag   611.000000  25.0000  25.0000
2515  Default   551.000000           plague   551.000000  24.0000  24.0000
1034  Default   484.000000             draw   484.000000  23.0000  23.0000
481   Default   400.000000       california   400.000000  22.0000  22.0000
1823  Default   373.000000       jacqueline   373.000000  21.0000  21.0000
2391  Default   379.000000        overprice   379.000000  20.0000  20.0000
462   Default   352.000000  burnsincomplete   352.000000  19.0000  19.0000
2684  Default   320.000000           racist   320.000000  18.0000  18.0000
1291  Default   304.000000      flabbergast   304.000000  17.0000  17.0000
80    Default   319.000000             aged   319.000000  16.0000  16.0000
2675  Default   310.000000            quiet   310.000000  15.0000  15.0000
211   Default   286.000000         assuming   286.000000  14.0000  14.0000
1491  Default   252.000000              gsw   252.000000  13.0000  13.0000
977   Default   245.000000          display   245.000000  12.0000  12.0000
416   Default   260.000000            bread   260.000000  11.0000  11.0000
1433  Default   236.000000             glee   236.000000  10.0000  10.0000
2769  Default   218.000000          regular   218.000000   9.0000   9.0000
2436  Default   211.000000         pathetic   211.000000   8.0000   8.0000
1023  Default   217.000000        downgrade   217.000000   7.0000   7.0000
1456  Default   210.000000       graduation   210.000000   6.0000   6.0000
172   Default   186.000000          approve   186.000000   5.0000   5.0000
1441  Default   198.000000              god   198.000000   4.0000   4.0000
1650  Default   187.000000        horseshoe   187.000000   3.0000   3.0000
2187  Default   186.000000           mollie   186.000000   2.0000   2.0000
1980  Default   180.000000           linear   180.000000   1.0000   1.0000
...       ...          ...              ...          ...      ...      ...
601   Topic10     0.000060           circus  1090.280884  -1.7840  -5.2956
1016  Topic10     0.000050           doozie   815.693665  -1.6606  -5.4624
1030  Topic10     0.000037             drag   611.999329  -1.6793  -5.7684
2515  Topic10     0.000036           plague   551.112793  -1.6114  -5.8053
2684  Topic10     0.000029           racist   320.244415  -1.2692  -6.0060
1034  Topic10     0.000033             draw   484.336029  -1.5702  -5.8933
1071  Topic10     0.000036             easy   694.922913  -1.8469  -5.8089
1823  Topic10     0.000028       jacqueline   373.728058  -1.4597  -6.0420
462   Topic10     0.000025  burnsincomplete   352.993439  -1.5363  -6.1757
2391  Topic10     0.000023        overprice   379.818787  -1.6988  -6.2649
481   Topic10     0.000022       cal